In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random

In [ ]:
class grid():
    def __init__(self, n_x, n_y):
        self.n_x = n_x
        self.n_y = n_y
        self.x_points = np.empty(0)
        self.y_points = np.empty(0)
        self.x_max = 1000
        self.y_max = 1000
        self.grid = {}  
        self.elevation = {}
        self.grid_customers = {}  
        self.n_customers = 0      
        self.customer_points = {}
        self.depot = None
        self.d_ij_indices = {}    
        self.d_ij = None          # distances among customers - interface output for the optimization model
        self.d_ij_extended = None
        self.output_matrix = None
        self.tw_a = None
        self.tw_b = None
        self.load = None
        self.servicetime = 5 # minutes
        
        self.create_grid()
        self.grid_dict()
        
    def create_grid(self):
        delta_x = 20
        delta_y = 20
        np.random.seed(1)
        self.x_points = np.linspace(0, self.x_max, self.n_x+1) + np.random.randint(-delta_x, delta_x, self.n_x+1)
        self.y_points = np.linspace(0, self.y_max, self.n_y+1) + np.random.randint(-delta_y, delta_y, self.n_y+1)
        self.x_points[0] = 0; self.x_points[-1] = self.x_max
        self.y_points[0] = 0; self.y_points[-1] = self.y_max
        self.x_points.sort()
        self.y_points.sort()
        
    def grid_dict(self):
        for i in range(self.n_x+1):
            for j in range(self.n_y+1):
                self.grid[(i,j)] = (self.x_points[i], self.y_points[j])
                self.elevation[(i,j)] = 30*np.random.random()
        return self.grid
    
    def time_windows(self, tp='random', min_t=50, max_t=200, k_separation = 0.05, k_overlapped = 0.5):
        tw_a = np.zeros(1, dtype=int); tw_b = np.zeros(1, dtype=int)
        if tp == 'random':
            delta_lower = min_t/2
            delta_upper = delta_lower
            tw_center = np.random.randint(min_t+delta_lower, max_t-delta_upper, self.n_customers)
            self.tw_a = np.append(tw_a, tw_center - np.random.randint(0, delta_lower, self.n_customers))
            self.tw_b = np.append(tw_b, tw_center + np.random.randint(0, delta_upper, self.n_customers))
            return self.tw_a, self.tw_b
        if tp == 'separated':
            delta_tw = (max_t - min_t)/self.n_customers
            self.tw_a = np.append(tw_a, np.array([min_t + (i - k_separation)*delta_tw for i in range(self.n_customers)]) )
            self.tw_b = np.append(tw_b, np.array([min_t + (i + k_separation)*delta_tw for i in range(self.n_customers)]) )
            return self.tw_a, self.tw_b
        if tp == 'overlapped':
            tw_c = (max_t - min_t)/2
            tw_interval = 0.5*k_overlapped*tw_c # half base interval (centered halfway between max_t and min_t)
            delta_interval = np.random.randint(0, tw_interval, self.n_customers)
            self.tw_a = np.append(tw_a, np.array([min_t + tw_c - t for t in sorted(delta_interval)]) )
            self.tw_b = np.append(tw_b, np.array([min_t + tw_c + t for t in sorted(delta_interval)]) )
            # np.random.shuffle(tw_a); np.random.shuffle(tw_b)
            return self.tw_a, self.tw_b
        if tp == 'single':
            self.tw_a = np.append(tw_a, np.array([min_t for _ in range(self.n_customers)]) )
            self.tw_b = np.append(tw_b, np.array([max_t for _ in range(self.n_customers)]) )
            return self.tw_a, self.tw_b
    
    def payload(self, min_p=5, max_p=30):
        self.load = np.append(np.zeros(1), np.random.randint(min_p, max_p, self.n_customers))
    
    def plot_grid(self):
        for i in range(self.n_x+1):
            plt.plot([self.x_points[i], self.x_points[i]], [0, self.y_max], color='gray')
        for j in range(self.n_y+1):
            plt.plot([0, self.x_max], [self.y_points[j], self.y_points[j]], color='gray')
        
    def customers(self, n_customers):
        self.n_customers = n_customers
        random.seed(10); customer_points = random.sample(list(self.grid.items()), n_customers)
        self.customer_points = {k:v for k, v in customer_points}
        return self.customer_points
        
    def plot_customer_points(self):
        k = list(self.customer_points.values())
        self.plot_grid()
        for i in range(self.n_customers):
            plt.scatter(k[i][0], k[i][1])

    def customers_paths(self, depot):
        ### depot must be an appropriate tuple of coordinate indices of the grid: ex (12, 15): (12th pos. on x - 15th pos. on y)
        self.depot = depot
        nodes = list(self.customer_points.keys())
        nodes.insert(0, depot)
        print(nodes)
        delta_indices = np.zeros((self.n_customers+1, self.n_customers+1), dtype=object)
        for i in range(self.n_customers+1):
            print('node:', i, nodes[i])
            for j in range(self.n_customers+1):
                dx_index = nodes[i][0] - nodes[j][0]
                self.d_ij_indices[(i,j)] = ((nodes[i][0], nodes[i][1]), (nodes[i][0]-dx_index, nodes[i][1]), (nodes[j][0], nodes[j][1]))
        
        self.d_ij = np.zeros((self.n_customers+1, self.n_customers+1))
        self.d_ij_extended = np.zeros((2*self.n_customers+1, 2*self.n_customers+1))
        
        for k, v in self.d_ij_indices.items():
            dx = abs(self.grid[v[1]][0] - self.grid[v[0]][0])
            dy = abs(self.grid[v[2]][1] - self.grid[v[1]][1])
            
            self.d_ij[k[0], k[1]] = dx + dy
        
        return self.d_ij
    
    def create_csv(self):
        elevation = np.zeros(self.n_customers)
        service_time = np.zeros(1)
        for i, k in enumerate(self.customer_points.keys()):
            elevation[i] = self.elevation[k]
        elevation = np.append(self.elevation[self.depot], elevation)
        service_time = np.append(service_time, np.array([self.servicetime for _ in range(self.n_customers)]) )
        x = np.append(self.grid[self.depot][0], np.array([v[0] for v in self.customer_points.values()]) )
        y = np.append(self.grid[self.depot][1], np.array([v[1] for v in self.customer_points.values()]) )
        index = np.linspace(0,self.n_customers,self.n_customers+1)
        
        self.output_matrix = np.c_[index, x, y, elevation, self.load, self.tw_a, self.tw_b, service_time, self.d_ij/1000]
        
        header = ',x,y,elevation,demand,tw a,tw b,s'
        for i in range(self.n_customers+1):
            header += ','+str(i)
            
        np.savetxt('generated_grid_1.csv', self.output_matrix, delimiter=',', header=header, comments='')
        
        return self.output_matrix

In [ ]:
G = grid(20, 20)
print(G.grid[(1,3)])
print(G.grid[(5,3)])

In [ ]:
np.linspace(0,21,22)

In [ ]:
np.append(np.zeros(1), np.random.randint(5, 10, 5))

In [ ]:
a,b = G.time_windows(tp='single', min_t= 50, max_t=400)
print(a)
print(b)

In [ ]:
G.plot_grid()

In [ ]:
print(np.round(G.x_points, 2))
print(np.round(G.y_points, 2))
print(G.n_x)

In [ ]:
print(G.elevation[(0,0)])
print(G.elevation[(0,5)])

In [ ]:
G.customers(20)
# for i in range(2,5): print(i)

In [ ]:
# G.customer_points

In [ ]:
G.plot_customer_points()

In [ ]:
G.customers_paths((0,0))

In [ ]:
# G.d_ij_indices

In [ ]:
# G.d_ij

In [ ]:
# print(G.grid_dict()[(0,0)])
# print(G.grid_dict()[(1,7)])
# print(G.grid_dict()[(16,23)])
# print(G.grid_dict()[(19,0)])

In [ ]:
# G.customer_points

In [ ]:
# for k in G.customer_points.keys():
#     print(k, ' elevation: ', G.elevation[k])

In [ ]:
ta, tb = G.time_windows()
print(ta)
print(tb)

In [ ]:
G.payload()

In [ ]:
out = G.create_csv()

In [ ]:
G1 = grid(10, 10)

In [ ]:
G1.customers(20)

In [ ]:
G1.plot_customer_points()

In [ ]:
G1.customers_paths((0,0))

In [ ]:
G1.time_windows()

In [ ]:
G1.payload()

In [ ]:
G1.create_csv()